## **Setup**
Click the Play Button. No need to expand. Do not touch this.

In [2]:
#@title Install VCD
!git clone https://github.com/yne/vcd.git
!make -C /content/vcd
%cd /content/vcd
!make install
%cd /

Cloning into 'vcd'...
remote: Enumerating objects: 156, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 156 (delta 25), reused 26 (delta 13), pack-reused 102
Receiving objects: 100% (156/156), 253.88 KiB | 769.00 KiB/s, done.
Resolving deltas: 100% (60/60), done.
make: Entering directory '/content/vcd'
cc     vcd.c   -o vcd
make: Leaving directory '/content/vcd'
/content/vcd
install -v -D vcd /usr/bin/vcd
'vcd' -> '/usr/bin/vcd'
/


In [3]:
#@title Set Up Widgets
import ipywidgets as widgets
from ipywidgets import GridspecLayout
from ipywidgets import AppLayout, Button, Layout, jslink, IntText, IntSlider

In [4]:
#@title Import Files from Repo
!mkdir -p /content/tmp_code
import requests
#Creates a text document using the Raw github url. This text document will be used as a .py file for imports
def import_text(text):
  url = "https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Tutorials/using_simulation_tools/files/%s" % text
  resp = requests.get(url)
  with open(text, 'wb') as f:
    f.write(resp.content)

import_text("import_all.py")
from import_all import *
import_source()
import_packages()
from simulation import *
from convertData import *

In [5]:
#@title Install Verilator
!apt-get install verilator >/dev/null

In [6]:
#@title Install Wavedrom Extension
!pip install --upgrade git+https://github.com/anon36424/nb_js_diagrammers.git
%load_ext nb_js_diagrammers

  Cloning https://github.com/anon36424/nb_js_diagrammers.git to /tmp/pip-req-build-v3dketvn
  Running command git clone --filter=blob:none --quiet https://github.com/anon36424/nb_js_diagrammers.git /tmp/pip-req-build-v3dketvn
  Resolved https://github.com/anon36424/nb_js_diagrammers.git to commit e5c241f9b76d69134b22ec2b1ed44023ee3e8443
  Preparing metadata (setup.py) ... done
  Created wheel for nb-js-diagrammers: filename=nb_js_diagrammers-0.0.7-py3-none-any.whl size=8897 sha256=c82a2468e5acfdddaae193ac18177a7e6d41a197769b11200891facf7e53ca80
  Stored in directory: /tmp/pip-ephem-wheel-cache-in7u83tt/wheels/cc/c5/ef/d8282dff3e12a74738cdebdae6a2aad9edccbdfed657ff4839
Successfully built nb-js-diagrammers


## **Simulation**



Simulating your hardware is an important part in the hardware design process. Making a module with SystemVerilog is very different than writing a program in C++ or Python. With SystemVerilog, for instance, we describe the hardware configuration that we want to implement, rather than giving a list of instructions for a processor to execute. After designing the hardware in SystemVerilog, there is still a lot more that needs to be done for it to be downloaded to an FPGA board, these steps include synthesis, placing and routing, and bitstream generation. These final steps perform a detailed analysis on and optimize the hardware implementation of your design, and generally take a fairly long time. If you need to make a change in your design or test it with some sample input, it would waste a lot of time to complete the whole process just to see if it works. We use simulations to reduce the amount of time it takes testing and visualizing the designed hardware.

### **Workspace**

Below is an image of the simulation workspace with each feature labeled.

<p align="left">
<img src="https://raw.githubusercontent.com/byuccl/digital_design_colab2/master/Tutorials/using_simulation_tools/media/SimWorkSpace.png"
width="500" height="" style="display: block; margin: 0 auto" />
</p>


1. **Source code area:** The SystemVerilog hardware design will be written in this textbox. It will contain the modules used, all logic, gates, and other SystemVerilog code that will be part of the hardware being designed. Quick changes to the system can be made here in the event of bugs or other unexpected results.

2. **Simulation code area:** The input to test your hardware design can be found and changed here. By changing what is inside this text box area, you will change the value of the input logics in the hardware design. The format is quite simple: The first line will be either  `comb` or `clk`. Use `comb` for simple combinational circuits and `clk` if the design uses timing and registers. The second line will have the name of each input signal separated by spaces. These are the signals that you will change to get different results in the simulation. All lines after the first two are a list of values that you want to assign to the input signals, separated by spaces. It can help to think of the whole simulation code area as a table with spaces instead of lines. The first line is the type of input, the second the names of the different input signals, and every row after that the values for each input signal.

3. **Save code button:** Click this button to save your code before running your simulation. It will save both the `Source code` and `Simulation code` areas to the current runtime of the Notebook. However, if you close the notebook and come back later, it will reload the default version of the files. Saving it to the runtime allows you to restore the code saved if, for instance, you want to try a different approach.

4. **Restore code button:** Click this button to restore your design files to the last saved version. If you press this button before saving, it will restore the default version of the files. Each time you press `Save code` it will overwrite the design to be restored. For example: if you press `Save code`, make a few changes, and then decide that these changes don't work, you can press `Restore code` to go back the design from when you pressed `Save code`.

5. **Select Simulation Input Type dropdown:** A dropdown menu to select the number system that you will be using in your `Simulation code` area. You can select between decimal, hexadecimal, and binary.

6. **Show source code below waveform selection:** Select and deselect this box to print out the source code below the text display of the simulation waveform. This is useful for double checking that the changes to your code were saved before simulation.

7. **Run simulation button:** Click this button to run the simulation. It will print out a waveform if the simulation was successful, or it will print out an error message if something went wrong. It is good practice to always click the `Save Code` button (3) before running the simulation.

### **Wavedrom Display**

After running the simulation, you can display your waveform again using the wavedrom tool. This tool produces an image that is easier to look at than the default output from the simulation workspace. To produce the wavedrom display, you just need to run the 2 code blocks that follow the simulation workspace widget. First press play on the "Create Wavedrom" block, and then press play on the "Show Wavedrom" block.

## **Practice**

We have given you a sample design so you can practice using the simulation tools. This design has 3 inputs: sw1, sw2, and btnc. sw1 and sw2 are each a set of 4 switches, and btnc is a button. If btnc is pressed then a bitwise and will be performed on sw1 and sw2 and the result will be displayed on some leds. Try writing a simulation sciript that will test this module, there are prompts for each line of the .stm file.

In [7]:
#@title Simulation Workspace
createSimulationWorkSpace("tmp_code/bitwiseAnd")

tmp_code/bitwiseAnd


In [ ]:
#@title Create WaveDrom
df2wd("bitwiseAnd")

In [ ]:
#@title Show WaveDrom
%%wavedrom_magic -h 200 -o /content/tmp_code/bitwiseAnd.txt
---